# Structural Simulation Exercise

Use your knowledge to instantiate the blocks in a module named "ALU".
Click the ▷ button below to setup the exercise's environment (will take about 2 mins).


In [11]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment

import os
import pathlib
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     --channel conda-forge \
                     iverilog \
                     verilator
!python -m pip install numpy
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}


Empty environment created at prefix: /content/conda-env

Pinned packages:

  - python==3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - iverilog
   - verilator


  Package                                             Version  Build            Channel         Size
──────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex                                         0.1  main             main          Cached
  + _openmp_mutex                                         5.1  1_gnu            main          Cached
  + _sysroot_linux-64_curr_repodata_hack                    3  haa98f57_10      main          Cached
  + binutils_impl_linux-64                               2.40  h5293946_0       main          Cached
  + binutils_linux-64                                  2.40.0  hc2dff05_2       main      

In [18]:
#@title Click the ▷ button to setup testbench
#@markdown The testbench allows us to check your work. No need to pay attention to this for now.

%%writefile tb_alu.v

`timescale 1ns/1ps

module tb_alu;
    // Parameters
    parameter WIDTH = 4;
    parameter CLK_PERIOD = 10;

    // DUT signals
    reg [WIDTH-1:0] a, b;
    reg [2:0] opcode;
    reg cin;
    wire [WIDTH-1:0] result;
    wire cout;
    wire zero;

    // Clock for timing (not used by DUT but useful for waveforms)
    reg clk;
    initial clk = 0;
    always #(CLK_PERIOD/2) clk = ~clk;

    // Instantiate DUT
    ALU #(.WIDTH(WIDTH)) dut (
        .a(a),
        .b(b),
        .opcode(opcode),
        .cin(cin),
        .result(result),
        .cout(cout),
        .zero(zero)
    );

    // Test counters
    integer test_count;
    integer pass_count;
    integer fail_count;

    // Operation name (ASCII string stored in reg array)
    reg [256*8-1:0] operation_name;

    // Test task
    task test_operation;
        input [WIDTH-1:0] test_a;
        input [WIDTH-1:0] test_b;
        input [2:0] test_opcode;
        input test_cin;
        input [WIDTH-1:0] expected_result;
        input expected_cout;
        input expected_zero;
        input [256*8-1:0] op_name;
    begin
        test_count = test_count + 1;

        // Apply inputs
        a = test_a;
        b = test_b;
        opcode = test_opcode;
        cin = test_cin;
        operation_name = op_name;

        // Wait for combinational logic to settle
        #1;

        // Check results
        if (result === expected_result && cout === expected_cout && zero === expected_zero) begin
            $display("PASS: %s | a=%b, b=%b, opcode=%b, cin=%b | result=%b, cout=%b, zero=%b",
                    operation_name, test_a, test_b, test_opcode, test_cin, result, cout, zero);
            pass_count = pass_count + 1;
        end else begin
            $display("FAIL: %s | a=%b, b=%b, opcode=%b, cin=%b",
                    operation_name, test_a, test_b, test_opcode, test_cin);
            $display("     Expected: result=%b, cout=%b, zero=%b",
                    expected_result, expected_cout, expected_zero);
            $display("     Got:      result=%b, cout=%b, zero=%b",
                    result, cout, zero);
            fail_count = fail_count + 1;
        end
    end
    endtask

    initial begin
        $display("=== ALU Structural Testbench ===");
        $display("Testing %d-bit ALU implementation", WIDTH);
        $display("");

        // Initialize
        a = 0; b = 0; opcode = 0; cin = 0;
        test_count = 0; pass_count = 0; fail_count = 0;
        #10;

        $display("--- Arithmetic Operations ---");
        // Test Addition (opcode 000)
        test_operation(4'b0001, 4'b0010, 3'b000, 1'b0, 4'b0011, 1'b0, 1'b0, "ADD: 1+2");
        test_operation(4'b1111, 4'b0001, 3'b000, 1'b0, 4'b0000, 1'b1, 1'b1, "ADD: 15+1 (overflow)");
        test_operation(4'b0101, 4'b1010, 3'b000, 1'b1, 4'b0000, 1'b1, 1'b1, "ADD: 5+10+1 (carry in)");

        // Test Subtraction (opcode 001)
        test_operation(4'b0101, 4'b0010, 3'b001, 1'b1, 4'b0100, 1'b1, 1'b0, "SUB: 5-2");
        test_operation(4'b0000, 4'b0001, 3'b001, 1'b1, 4'b0000, 1'b1, 1'b1, "SUB: 0-1 (zero result)");

        $display("");
        $display("--- Logical Operations ---");
        test_operation(4'b1100, 4'b1010, 3'b010, 1'b0, 4'b1000, 1'b0, 1'b0, "AND: 1100 & 1010");
        test_operation(4'b1111, 4'b0000, 3'b010, 1'b0, 4'b0000, 1'b0, 1'b1, "AND: 1111 & 0000");

        test_operation(4'b1100, 4'b1010, 3'b011, 1'b0, 4'b1110, 1'b0, 1'b0, "OR: 1100 | 1010");
        test_operation(4'b0000, 4'b0000, 3'b011, 1'b0, 4'b0000, 1'b0, 1'b1, "OR: 0000 | 0000");

        $display("");
        $display("--- Shift Operations ---");
        test_operation(4'b0011, 4'b0000, 3'b100, 1'b0, 4'b0110, 1'b0, 1'b0, "SHL: 0011 << 1");
        test_operation(4'b1000, 4'b0000, 3'b100, 1'b0, 4'b0000, 1'b1, 1'b1, "SHL: 1000 << 1 (overflow)");

        test_operation(4'b1100, 4'b0000, 3'b101, 1'b0, 4'b0110, 1'b0, 1'b0, "SHR: 1100 >> 1");
        test_operation(4'b0001, 4'b0000, 3'b101, 1'b0, 4'b0000, 1'b0, 1'b1, "SHR: 0001 >> 1 (zero)");

        $display("");
        $display("--- XOR and NOT Operations ---");
        test_operation(4'b1100, 4'b1010, 3'b110, 1'b0, 4'b0110, 1'b0, 1'b0, "XOR: 1100 ^ 1010");
        test_operation(4'b1111, 4'b1111, 3'b110, 1'b0, 4'b0000, 1'b0, 1'b1, "XOR: 1111 ^ 1111");

        test_operation(4'b1010, 4'b0000, 3'b111, 1'b0, 4'b0101, 1'b0, 1'b0, "NOT: ~1010");
        test_operation(4'b1111, 4'b0000, 3'b111, 1'b0, 4'b0000, 1'b0, 1'b1, "NOT: ~1111");

        $display("");
        $display("--- Edge Cases ---");
        test_operation(4'b0000, 4'b0000, 3'b000, 1'b0, 4'b0000, 1'b0, 1'b1, "Edge: All zeros");
        test_operation(4'b1111, 4'b1111, 3'b000, 1'b0, 4'b1110, 1'b1, 1'b0, "Edge: All ones ADD");

        $display("");
        $display("=== Test Summary ===");
        $display("Total Tests: %0d", test_count);
        $display("Passed:      %0d", pass_count);
        $display("Failed:      %0d", fail_count);

        if (fail_count == 0) begin
            $display("ALL TESTS PASSED!");
        end else begin
            $display("Some tests failed. Check your ALU implementation.");
        end

        $finish;
    end

    // Generate VCD for waveform viewing
    initial begin
        $dumpfile("alu_test.vcd");
        $dumpvars(0, tb_alu);
    end

endmodule


Overwriting tb_alu.v


In [26]:
#@title Click to write components
#@markdown This generates the components you can instantiate structurally. No need to pay attention to this.

%%writefile components.v

module full_adder (
    input  a, b, cin,
    output sum, cout
);
    assign sum  = a ^ b ^ cin;
    assign cout = (a & b) | (a & cin) | (b & cin);
endmodule

module ripple_carry_adder #(parameter WIDTH = 4) (
    input  [WIDTH-1:0] a, b,
    input  cin,
    output [WIDTH-1:0] sum,
    output cout
);
    wire [WIDTH:0] carry;
    assign carry[0] = cin;
    assign cout = carry[WIDTH];

    genvar i;
    generate
        for (i = 0; i < WIDTH; i = i + 1) begin : adder_chain
            full_adder fa_inst (
                .a(a[i]),
                .b(b[i]),
                .cin(carry[i]),
                .sum(sum[i]),
                .cout(carry[i+1])
            );
        end
    endgenerate
endmodule

module logical_unit #(parameter WIDTH = 4) (
    input  [WIDTH-1:0] a, b,
    input  [1:0] op,
    output reg [WIDTH-1:0] result
);

    always @(*) begin
        case (op)
            2'b00: result = a & b;   // AND
            2'b01: result = a | b;   // OR
            2'b10: result = a ^ b;   // XOR
            2'b11: result = ~a;      // NOT
        endcase
    end
endmodule

module shifter #(parameter WIDTH = 4) (
    input  [WIDTH-1:0] data,
    input  [1:0] shift_op,
    output reg [WIDTH-1:0] result
);

    always @(*) begin
        case (shift_op)
            2'b00: result = data;                        // No shift
            2'b01: result = data << 1;                   // Left shift
            2'b10: result = data >> 1;                   // Right shift
            2'b11: result = {data[0], data[WIDTH-1:1]};  // Rotate right
        endcase
    end
endmodule

Overwriting components.v


# The exercise

Fill in your verilog code below:

In [30]:
%%writefile alu.v

module ALU #(parameter WIDTH = 4) (
    input [WIDTH-1:0] a, b,
    input [2:0] opcode,
    input cin,
    output [WIDTH-1:0] result,
    output cout,
    output zero
);

    // Wires for submodules
    wire [WIDTH-1:0] add_sum, sub_sum;
    wire add_cout, sub_cout;
    wire [WIDTH-1:0] logic_out;
    wire [WIDTH-1:0] shifter_out;

    // Arithmetic: ADD
    ripple_carry_adder #(WIDTH) add_inst (
        .a(a),
        .b(b),
        .cin(cin),
        .sum(add_sum),
        .cout(add_cout)
    );

    // Arithmetic: SUB (a + ~b + 1)
    wire [WIDTH-1:0] b_inverted;
    assign b_inverted = ~b;

    ripple_carry_adder #(WIDTH) sub_inst (
        .a(a),
        .b(b_inverted),
        .cin(1'b1),      // carry-in = 1 for two’s complement subtraction
        .sum(sub_sum),
        .cout(sub_cout)
    );

    // Logical unit (AND, OR, XOR, NOT)
    // Map opcode[1:0] to the operations:
    //   00 = AND
    //   01 = OR
    //   10 = XOR
    //   11 = NOT
    logical_unit #(WIDTH) logic_inst (
        .a(a),
        .b(b),
        .op(opcode[1:0]),
        .result(logic_out)
    );

    // Shifter (opcode[0] selects left/right, opcode[1] ignored)
    // For this ALU:
    //   opcode=100 → shift left
    //   opcode=101 → shift right
    // We'll reuse shift_op = {1'b0, opcode[0]}
    shifter #(WIDTH) shifter_inst (
        .data(a),
        .shift_op({1'b0, opcode[0]}), // 01=left, 10=right
        .result(shifter_out)
    );

    // Result mux
    reg [WIDTH-1:0] result_reg;
    reg cout_reg;

    always @(*) begin
        case (opcode)
            3'b000: begin // ADD
                result_reg = add_sum;
                cout_reg   = add_cout;
            end
            3'b001: begin // SUB
                result_reg = sub_sum;
                cout_reg   = sub_cout;
            end
            3'b010,
            3'b011,
            3'b110,
            3'b111: begin // Logic ops
                result_reg = logic_out;
                cout_reg   = 1'b0;
            end
            3'b100,
            3'b101: begin // Shifts
                result_reg = shifter_out;
                cout_reg   = (opcode == 3'b100) ? a[WIDTH-1] : a[0]; // capture shifted-out bit
            end
            default: begin
                result_reg = {WIDTH{1'b0}};
                cout_reg   = 1'b0;
            end
        endcase
    end

    assign result = result_reg;
    assign cout   = cout_reg;
    assign zero   = (result_reg == {WIDTH{1'b0}});

endmodule

Overwriting alu.v


# Checking

Press the ▷ button to check your work

In [31]:
%cd content
! iverilog tb_alu.v alu.v components.v
! ./a.out

[Errno 2] No such file or directory: 'content'
/content
=== ALU Structural Testbench ===
Testing           4-bit ALU implementation

VCD info: dumpfile alu_test.vcd opened for output.
--- Arithmetic Operations ---
PASS:                                                                                                                                                                                                                                                         ADD: 1+2 | a=0001, b=0010, opcode=000, cin=0 | result=0011, cout=0, zero=0
PASS:                                                                                                                                                                                                                                             ADD: 15+1 (overflow) | a=1111, b=0001, opcode=000, cin=0 | result=0000, cout=1, zero=1
PASS:                                                                                                                           